## Week 6: File ingestion and schema validation

* Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

* Read the file ( Present approach of reading the file )

* Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational     efficiency

* Perform basic validation on data columns : eg: remove special character , white spaces from the col name

* As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of   
  read and write file, column name in YAML

* Validate number of columns and column name of ingested file with YAML.

* Write the file in pipe separated text file (|) in gz format.

* Create a summary of the file:

    Total number of rows,

    total number of columns

    file size

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import time

In [8]:
#Size of the file
os.path.getsize('/content/drive/MyDrive/Intern week6/Books_rating.csv')

2859504349

### Read in the data with Dask

In [9]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('/content/drive/MyDrive/Intern week6/Books_rating.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.2910168170928955 sec


### Read in the data with Pandas

In [10]:
import pandas as pd
start = time.time()
df = pd.read_csv('/content/drive/MyDrive/Intern week6/Books_rating.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  67.73333406448364 sec


### Read in the data with Modin and Ray

In [11]:
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 956 kB 5.2 MB/s 
     |████████████████████████████████| 761 kB 41.2 MB/s 


In [13]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 58 kB/s 
     |████████████████████████████████| 8.8 MB 43.2 MB/s 
     |████████████████████████████████| 4.1 MB 34.0 MB/s 
     |████████████████████████████████| 468 kB 66.9 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.49.1
    Uninstalling grpcio-1.49.1:
      Successfully uninstalled grpcio-1.49.1


In [15]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('/content/drive/MyDrive/Intern week6/Books_rating.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2022-10-07 01:25:53,573	INFO worker.py:1518 -- Started a local Ray instance.
(deploy_ray_func pid=903) tcmalloc: large alloc 1429757952 bytes == 0x33d6000 @  0x7fdac6d911e7 0x4b2150 0x5ac2ec 0x5e3d6d 0x58ee7f 0x51c455 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x4ba80a 0x51908c 0x5b41c5 0x4ba80a 0x51908c 0x4ba70a 0x7fdac3a86b92 0x7fdac3b1c0f8 0x7fdac3a8d3b6 0x7fdac3c0d44d 0x7fdac3b6ccaf 0x7fdac3c367d3 0x7fdac3c376ca 0x7fdac3c4b95e 0x7fdac3c21bb0 0x7fdac3e45796 0x7fdac3dfe5fe 0x7fdac3dfe856 0x7fdac427667b 0x7fdac42778b1 0x7fdac4277b20
(deploy_ray_func pid=902) tcmalloc: large alloc 1429757952 bytes == 0x5026000 @  0x7f0e220091e7 0x4b2150 0x5ac2ec 0x5e3d6d 0x58ee7f 0x51c455 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x4ba80a 0x51908c 0x5b41c5 0x4ba80a 0x51908c 0x4ba70a 0x7f0e1ecfeb92 0x7f0e1ed940f8 0x7f0e1ed053b6 0x7f0e1ee8544d 0x7f0e1ede4caf 0x7f0e1eeae7d3 0x7f0e1eeaf6ca 0x7f0e1eec395e 0x7f0e1ee99bb0 0x7f0e1f0bd796 0x7f0e1f0765fe 0x7f0e1f076856 0x7f0e1f4ee67b 0x7f0e1f4ef8b1 0x7f0e1f4efb20
(deploy_ray

Read csv with modin and ray:  210.13118863105774 sec


### Here Dask is better than Pandas, Modin and Ray, with the least reading time of 0.29 sec

In [19]:
from dask import dataframe
df = dataframe.read_csv('/content/drive/MyDrive/Intern week6/Books_rating.csv',delimiter=',', dtype={'Id': 'object'})

In [20]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 10 entries, Id to review/text
dtypes: object(7), float64(2), int64(1)

In [21]:
#No. of Rows
len(df.index)

3000000

In [22]:
#No, of Columns
len(df.columns)

10

In [23]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [24]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [25]:
data=df.columns
data

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

### Validation

In [28]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [47]:
config_data = read_config_file("/content/drive/MyDrive/Intern week6/store.yaml")

In [48]:
#data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'Books_rating',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Id',
  'Title',
  'Price',
  'User_id',
  'profileName',
  'review_helpfulness',
  'review_score',
  'review_time',
  'review_summary',
  'review_text']}

In [42]:
# Reading process of the file using Dask
from dask import dataframe
df_sample = dataframe.read_csv('/content/drive/MyDrive/Intern week6/Books_rating.csv',delimiter=',', dtype={'Id': 'object'})
df_sample.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [49]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = '/content/drive/MyDrive/Intern week6/' + config_data['file_name'] + f'.{file_type}'

In [50]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [51]:
#validating the header of the file
col_header_val(df,config_data)

column name and column length validation passed


1

In [52]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['id', 'title', 'price', 'user_id', 'profilename', 'review_helpfulness',
       'review_score', 'review_time', 'review_summary', 'review_text'],
      dtype='object')
columns of YAML are: ['Id', 'Title', 'Price', 'User_id', 'profileName', 'review_helpfulness', 'review_score', 'review_time', 'review_summary', 'review_text']


In [53]:
if col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation passed
col validation passed


In [55]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('/content/drive/MyDrive/Intern week6/Books_rating.csv',delimiter=',', dtype={'Id': 'object'})

# Write csv in gz format in pipe separated text file (|)
df.to_csv('/content/drive/MyDrive/Intern week6/Books_rating.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/00.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/01.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/02.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/03.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/04.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/05.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/06.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/07.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/08.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/09.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/10.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/11.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/12.part',
 '/content/drive/MyDrive/Intern week6/Books_rating.csv.gz/13.part',
 '/content/drive/MyDrive/Intern week6/Books_rati

In [56]:
#number of files in gz format folder
import os
entries = os.listdir('/content/drive/MyDrive/Intern week6/Books_rating.csv.gz')
for entry in entries:
    print(entry)

01.part
00.part
02.part
03.part
04.part
05.part
06.part
07.part
08.part
09.part
10.part
11.part
12.part
13.part
14.part
15.part
16.part
17.part
18.part
19.part
20.part
21.part
23.part
22.part
24.part
25.part
26.part
27.part
28.part
29.part
30.part
31.part
33.part
32.part
34.part
35.part
36.part
37.part
38.part
39.part
40.part
41.part
42.part
43.part


In [57]:
#size of the gz format folder
os.path.getsize('/content/drive/MyDrive/Intern week6/Books_rating.csv.gz')

4096